In [1]:
# header files
%matplotlib inline
import torch
import numpy as np
import csv
import glob
import math
import tensorboard
from PIL import Image
import skimage
from skimage import io, transform
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [15, 15]
print("Header files loaded!")

Header files loaded!


### Description of the notebook
This notebook was used for some analysis on the annotations that were obtained from Broad (obtained 11 csvs). However, now a much better alternative is there, where we can use a single file namely, qc_features_annotations_full.parquet can be used as seen in the read_annotation_data.ipynb notebook.

In [2]:
files = glob.glob("/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/qc_data/data/qc_1/*")
print(files)
print(len(files))

['/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/qc_data/data/qc_1/Classifier_20201020T134356__David.csv', '/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/qc_data/data/qc_1/Classifier_Batch12_Erin.csv', '/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/qc_data/data/qc_1/Classifier_PE_NC_Bin2_Pearl.csv', '/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/qc_data/data/qc_1/Classifier_Batch10_Erin.csv', '/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/qc_data/data/qc_1/Classifier_Batch13_Erin.csv', '/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/qc_data/data/qc_1/Classifier_Batch11_Erin.csv', '/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/qc_data/data/qc_1/Classifier_C_BIN2__David.csv', '/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/qc_data/data/qc_1/Classifier_NC_BIN1_Erin.csv', '/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/qc_data/data/qc_1/Classifier_Plate2-PCO-6ch-4site-10XPA_David.csv', '

In [3]:
# read csv files and obtain frequency for each class
array_labels = []
array_image_path = []
array_image_size = []
array_image_path_label_1 = []
array_image_size_label_1 = []
array_image_path_label_0 = []
array_image_size_label_0 = []
array_image_path_label_2 = []
array_image_size_label_2 = []
count_good = 0
count_blurry = 0
count_empty = 0

files = [files[7]]
for file in files:
    print(file)
    
    flag = -1
    flag_good = 1
    flag_debris = -1
    flag_empty = -1
    flag_saturated = -1
    flag_blurry = -1
    flag_dead = -1
    flag_halo = -1
    flag_image_width = -1
    flag_image_height = -1
    flag_image_path_dna = -1
    flag_image_path_rna = -1
    flag_image_path_er = -1
    flag_image_path_mito = -1
    flag_image_path_agp = -1
    flag_image_path_golgi = -1
    flag_image_path_actin = -1
    
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                array = row
                if "CPAScore_ScratchImageCount" in array:
                    for index in range(0, len(array)):
                        if array[index] == "CPAScore_ScratchImageCount":
                            flag_saturated = index
                if "CPAScore_BlurryImageCount" in array:
                    for index in range(0, len(array)):
                        if array[index] == "CPAScore_BlurryImageCount":
                            flag_blurry = index
                if "CPAScore_DebrisImageCount" in array:
                    for index in range(0, len(array)):
                        if array[index] == "CPAScore_DebrisImageCount":
                            flag_debris = index
                if "CPAScore_EmptyImageCount" in array:
                    for index in range(0, len(array)):
                        if array[index] == "CPAScore_EmptyImageCount":
                            flag_empty = index
                if "Image_Width_OrigDNA" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_Width_OrigDNA":
                            flag_image_width = index
                if "Image_Height_OrigDNA" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_Height_OrigDNA":
                            flag_image_height = index
                if "Image_URL_OrigDNA" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigDNA":
                            flag_image_path_dna = index
                if "Image_URL_OrigRNA" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigRNA":
                            flag_image_path_rna = index
                if "Image_URL_OrigER" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigER":
                            flag_image_path_er = index
                if "Image_URL_OrigMito" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigMito":
                            flag_image_path_mito = index
                if "Image_URL_OrigAGP" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigAGP":
                            flag_image_path_agp = index
                if "Image_URL_OrigActin" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigActin":
                            flag_image_path_actin = index
                if "Image_URL_OrigGolgi" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigGolgi":
                            flag_image_path_golgi = index
                flag = 1
                print(array)
            else:
                array = row
                print(array)
                break
                
                # check multi-class and remove them
                check_multi_class = 0
                check = 0
                if flag_blurry != -1 and float(array[flag_blurry]) > 0:
                    check_multi_class += 1
                if flag_debris != -1 and float(array[flag_debris]) > 0:
                    check_multi_class += 1
                    check = 1
                if flag_saturated != -1 and float(array[flag_saturated]) > 0:
                    check_multi_class += 1
                if flag_empty != -1 and float(array[flag_empty]) > 0:
                    check_multi_class += 1
                if flag_dead != -1 and float(array[flag_dead]) > 0:
                    check_multi_class += 1
                if flag_halo != -1 and float(array[flag_halo]) > 0:
                    check_multi_class += 1
                if check_multi_class > 1:
                    continue
                
                check = -1
                is_saturated = -1
                label = 0
                if flag_blurry != -1 and float(array[flag_blurry]) > 0:
                    check += 1
                    label = 1
                if flag_debris != -1 and float(array[flag_debris]) > 0:
                    check += 1
                    label = 3
                if flag_saturated != -1 and float(array[flag_saturated]) > 0:
                    check += 1
                    is_saturated = 1
                if flag_halo != -1 and float(array[flag_halo]) > 0:
                    check += 1
                    is_saturated = 1
                if flag_empty != -1 and float(array[flag_empty]) > 0:
                    check += 1
                    label = 2
                if flag_dead != -1 and float(array[flag_dead]) > 0:
                    check += 1
                    label = 2
                    
                if is_saturated == -1 and label == 3:
                    continue
                    if flag_image_path_dna != -1:
                        array_image_debris_path.append(str(array[flag_image_path_dna]))
                        count_debris += 1
                    if flag_image_path_rna != -1:
                        array_image_debris_path.append(str(array[flag_image_path_rna]))
                        count_debris += 1
                    if flag_image_path_er != -1:
                        array_image_debris_path.append(str(array[flag_image_path_er]))
                        count_debris += 1
                    if flag_image_path_mito != -1:
                        array_image_debris_path.append(str(array[flag_image_path_mito]))
                        count_debris += 1
                    if flag_image_path_agp != -1:
                        array_image_debris_path.append(str(array[flag_image_path_agp]))
                        count_debris += 1
                    if flag_image_path_actin != -1:
                        array_image_debris_path.append(str(array[flag_image_path_actin]))
                        count_debris += 1
                    if flag_image_path_golgi != -1:
                        array_image_debris_path.append(str(array[flag_image_path_golgi]))
                        count_debris += 1
                    if flag_image_path_dna != -1 and flag_image_path_er != -1 and flag_image_path_mito != -1 and flag_image_path_actin != -1:
                        count_debris_4channel += 1
                elif is_saturated == -1:
                    if flag_image_path_dna != -1:
                        image = str(array[flag_image_path_dna])
                        if "Nikon" in image:
                            continue
                        
                        array_image_path.append(str(array[flag_image_path_dna]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        
                        if int(array[flag_image_height]) < 6200 and int(array[flag_image_height]) > 2000:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_dna]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_dna]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_dna]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                    if flag_image_path_rna != -1:
                        image = str(array[flag_image_path_rna])
                        if "Nikon" in image:
                            continue
                        
                        array_image_path.append(str(array[flag_image_path_rna]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        
                        if int(array[flag_image_height]) < 6200 and int(array[flag_image_height]) > 2000:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_rna]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_rna]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_rna]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                    if flag_image_path_er != -1:
                        image = str(array[flag_image_path_er])
                        if "Nikon" in image:
                            continue
                        
                        array_image_path.append(str(array[flag_image_path_er]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        
                        if int(array[flag_image_height]) < 6200 and int(array[flag_image_height]) > 2000:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_er]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_er]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_er]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                    if flag_image_path_mito != -1:
                        image = str(array[flag_image_path_mito])
                        if "Nikon" in image:
                            continue
                        
                        array_image_path.append(str(array[flag_image_path_mito]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        
                        if int(array[flag_image_height]) < 6200 and int(array[flag_image_height]) > 2000:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_mito]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_mito]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_mito]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                    if flag_image_path_agp != -1:
                        image = str(array[flag_image_path_agp])
                        if "Nikon" in image:
                            continue
                        
                        array_image_path.append(str(array[flag_image_path_agp]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        
                        if int(array[flag_image_height]) < 6200 and int(array[flag_image_height]) > 2000:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_agp]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_agp]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_agp]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                    if flag_image_path_actin != -1:
                        image = str(array[flag_image_path_actin])
                        if "Nikon" in image:
                            continue
                        
                        array_image_path.append(str(array[flag_image_path_actin]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        
                        if int(array[flag_image_height]) < 6200 and int(array[flag_image_height]) > 2000:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_actin]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_actin]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_actin]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                    if flag_image_path_golgi != -1:
                        image = str(array[flag_image_path_golgi])
                        if "Nikon" in image:
                            continue
                        
                        array_image_path.append(str(array[flag_image_path_golgi]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        
                        if int(array[flag_image_height]) < 6200 and int(array[flag_image_height]) > 2000:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_golgi]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_golgi]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_golgi]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                        
                    
print(len(array_image_path_label_0))
print(len(array_image_size_label_0))
print(len(array_image_path_label_1))
print(len(array_image_size_label_1))
print(len(array_image_path_label_2))
print(len(array_image_size_label_2))
print("Done processing!")

/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/qc_data/data/qc_1/Classifier_NC_BIN1_Erin.csv
['Image_ExecutionTime_01LoadData', 'Image_ExecutionTime_02MeasureImageQuality', 'Image_ExecutionTime_03MeasureImageQuality', 'Image_FileName_OrigDNA', 'Image_FileName_OrigER', 'Image_FileName_OrigMito', 'Image_FileName_OrigRNA', 'Image_FileName_OrigAGP', 'Image_Group_Index', 'Image_Group_Length', 'Image_Group_Number', 'Image_Height_OrigDNA', 'Image_Height_OrigER', 'Image_Height_OrigMito', 'Image_Height_OrigRNA', 'Image_Height_OrigAGP', 'ImageNumber', 'Image_ImageQuality_Correlation_OrigDNA_10', 'Image_ImageQuality_Correlation_OrigDNA_20', 'Image_ImageQuality_Correlation_OrigDNA_5', 'Image_ImageQuality_Correlation_OrigDNA_50', 'Image_ImageQuality_Correlation_OrigER_10', 'Image_ImageQuality_Correlation_OrigER_20', 'Image_ImageQuality_Correlation_OrigER_5', 'Image_ImageQuality_Correlation_OrigER_50', 'Image_ImageQuality_Correlation_OrigMito_10', 'Image_ImageQuality_Correlation_OrigMito_

In [4]:
# read csv files and obtain frequency for each class
array_labels = []
array_image_path = []
array_image_size = []
array_image_path_label_1 = []
array_image_size_label_1 = []
array_image_path_label_0 = []
array_image_size_label_0 = []
array_image_path_label_2 = []
array_image_size_label_2 = []
count_good = 0
count_blurry = 0
count_empty = 0
plate = "Plate3-PCO-6ch-4site-10XPA"
plate_files = []

for file in files:
    flag = -1
    flag_good = 1
    flag_debris = -1
    flag_empty = -1
    flag_saturated = -1
    flag_blurry = -1
    flag_dead = -1
    flag_halo = -1
    flag_image_width = -1
    flag_image_height = -1
    flag_image_path_dna = -1
    flag_image_path_rna = -1
    flag_image_path_er = -1
    flag_image_path_mito = -1
    flag_image_path_agp = -1
    flag_image_path_golgi = -1
    flag_image_path_actin = -1
    
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                array = row
                if "UserAnnotation_Scratch" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Scratch":
                            flag_saturated = index
                if "UserAnnotation_Halo" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Halo":
                            flag_halo = index
                if "UserAnnotation_Blurry" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Blurry":
                            flag_blurry = index
                if "UserAnnotation_Debris" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Debris":
                            flag_debris = index
                if "UserAnnotation_Empty" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Empty":
                            flag_empty = index
                if "UserAnnotation_Saturated" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Saturated":
                            flag_saturated = index
                if "UserAnnotation_Ded" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Ded":
                            flag_dead = index
                if "Image_Width_OrigDNA" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_Width_OrigDNA":
                            flag_image_width = index
                if "Image_Height_OrigDNA" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_Height_OrigDNA":
                            flag_image_height = index
                if "Image_URL_OrigDNA" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigDNA":
                            flag_image_path_dna = index
                if "Image_URL_OrigRNA" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigRNA":
                            flag_image_path_rna = index
                if "Image_URL_OrigER" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigER":
                            flag_image_path_er = index
                if "Image_URL_OrigMito" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigMito":
                            flag_image_path_mito = index
                if "Image_URL_OrigAGP" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigAGP":
                            flag_image_path_agp = index
                if "Image_URL_OrigActin" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigActin":
                            flag_image_path_actin = index
                if "Image_URL_OrigGolgi" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigGolgi":
                            flag_image_path_golgi = index
                flag = 1
            else:
                array = row
                
                # check multi-class and remove them
                check_multi_class = 0
                check = 0
                if flag_blurry != -1 and float(array[flag_blurry]) > 0:
                    check_multi_class += 1
                if flag_debris != -1 and float(array[flag_debris]) > 0:
                    check_multi_class += 1
                    check = 1
                if flag_saturated != -1 and float(array[flag_saturated]) > 0:
                    check_multi_class += 1
                if flag_empty != -1 and float(array[flag_empty]) > 0:
                    check_multi_class += 1
                if flag_dead != -1 and float(array[flag_dead]) > 0:
                    check_multi_class += 1
                if flag_halo != -1 and float(array[flag_halo]) > 0:
                    check_multi_class += 1
                if check_multi_class > 1:
                    continue
                
                check = -1
                is_saturated = -1
                label = 0
                if flag_blurry != -1 and float(array[flag_blurry]) > 0:
                    check += 1
                    label = 1
                if flag_debris != -1 and float(array[flag_debris]) > 0:
                    check += 1
                    label = 3
                if flag_saturated != -1 and float(array[flag_saturated]) > 0:
                    check += 1
                    is_saturated = 1
                if flag_halo != -1 and float(array[flag_halo]) > 0:
                    check += 1
                    is_saturated = 1
                if flag_empty != -1 and float(array[flag_empty]) > 0:
                    check += 1
                    label = 2
                if flag_dead != -1 and float(array[flag_dead]) > 0:
                    check += 1
                    label = 2
                    
                f = -1
                if is_saturated == -1 and label == 3:
                    continue
                    if flag_image_path_dna != -1:
                        array_image_debris_path.append(str(array[flag_image_path_dna]))
                        count_debris += 1
                    if flag_image_path_rna != -1:
                        array_image_debris_path.append(str(array[flag_image_path_rna]))
                        count_debris += 1
                    if flag_image_path_er != -1:
                        array_image_debris_path.append(str(array[flag_image_path_er]))
                        count_debris += 1
                    if flag_image_path_mito != -1:
                        array_image_debris_path.append(str(array[flag_image_path_mito]))
                        count_debris += 1
                    if flag_image_path_agp != -1:
                        array_image_debris_path.append(str(array[flag_image_path_agp]))
                        count_debris += 1
                    if flag_image_path_actin != -1:
                        array_image_debris_path.append(str(array[flag_image_path_actin]))
                        count_debris += 1
                    if flag_image_path_golgi != -1:
                        array_image_debris_path.append(str(array[flag_image_path_golgi]))
                        count_debris += 1
                    if flag_image_path_dna != -1 and flag_image_path_er != -1 and flag_image_path_mito != -1 and flag_image_path_actin != -1:
                        count_debris_4channel += 1
                if f == -1:
                    if flag_image_path_dna != -1:
                        image = str(array[flag_image_path_dna])
                        if "Nikon" in image or ".nd2" in image:
                            continue
                        if plate in image:
                            plate_files.append(image)
                        
                        
                        array_image_path.append(str(array[flag_image_path_dna]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        if label == 2:
                            count_empty += 1
                        
                        
                        if int(array[flag_image_height]) < 50000 and int(array[flag_image_height]) > 0:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_dna]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_dna]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                #count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_dna]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                    if flag_image_path_rna != -1:
                        image = str(array[flag_image_path_rna])
                        if "Nikon" in image or ".nd2" in image:
                            continue
                        if plate in image:
                            plate_files.append(image)
                        
                        array_image_path.append(str(array[flag_image_path_rna]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        if label == 2:
                            count_empty += 1
                        
                        if int(array[flag_image_height]) < 50000 and int(array[flag_image_height]) > 0:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_rna]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_rna]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                #count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_rna]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                    if flag_image_path_er != -1:
                        image = str(array[flag_image_path_er])
                        if "Nikon" in image or ".nd2" in image:
                            continue
                        if plate in image:
                            plate_files.append(image)
                        
                        array_image_path.append(str(array[flag_image_path_er]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        if label == 2:
                            count_empty += 1
                        
                        if int(array[flag_image_height]) < 50000 and int(array[flag_image_height]) > 0:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_er]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_er]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                #count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_er]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                    if flag_image_path_mito != -1:
                        image = str(array[flag_image_path_mito])
                        if "Nikon" in image or ".nd2" in image:
                            continue
                        if plate in image:
                            plate_files.append(image)
                        
                        array_image_path.append(str(array[flag_image_path_mito]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        if label == 2:
                            count_empty += 1
                        
                        if int(array[flag_image_height]) < 50000 and int(array[flag_image_height]) > 0:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_mito]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_mito]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                #count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_mito]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                    if flag_image_path_agp != -1:
                        image = str(array[flag_image_path_agp])
                        if "Nikon" in image or ".nd2" in image:
                            continue
                        if plate in image:
                            plate_files.append(image)
                        
                        array_image_path.append(str(array[flag_image_path_agp]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        if label == 2:
                            count_empty += 1
                        
                        if int(array[flag_image_height]) < 50000 and int(array[flag_image_height]) > 0:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_agp]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_agp]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                #count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_agp]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                    if flag_image_path_actin != -1:
                        image = str(array[flag_image_path_actin])
                        if "Nikon" in image or ".nd2" in image:
                            continue
                        if plate in image:
                            plate_files.append(image)
                        
                        array_image_path.append(str(array[flag_image_path_actin]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        if label == 2:
                            count_empty += 1
                        
                        if int(array[flag_image_height]) < 50000 and int(array[flag_image_height]) > 0:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_actin]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_actin]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                #count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_actin]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                    if flag_image_path_golgi != -1:
                        image = str(array[flag_image_path_golgi])
                        if "Nikon" in image or ".nd2" in image:
                            continue
                        if plate in image:
                            plate_files.append(image)
                        
                        array_image_path.append(str(array[flag_image_path_golgi]))
                        array_labels.append(str(label))
                        array_image_size.append(str(array[flag_image_height]))
                        if label == 2:
                            count_empty += 1
                        
                        if int(array[flag_image_height]) < 50000 and int(array[flag_image_height]) > 0:
                            if label == 0:
                                count_good += 1
                                array_image_path_label_0.append(str(array[flag_image_path_golgi]))
                                array_image_size_label_0.append(str(array[flag_image_height]))
                            if label == 1:
                                count_blurry += 1
                                array_image_path_label_1.append(str(array[flag_image_path_golgi]))
                                array_image_size_label_1.append(str(array[flag_image_height]))
                            if label == 2:
                                #count_empty += 1
                                array_image_path_label_2.append(str(array[flag_image_path_golgi]))
                                array_image_size_label_2.append(str(array[flag_image_height]))
                        
                    
print(len(array_image_path_label_0))
print(len(array_image_size_label_0))
print(len(array_image_path_label_1))
print(len(array_image_size_label_1))
print(len(array_image_path_label_2))
print(len(array_image_size_label_2))
print(count_empty)
print(len(plate_files))
print("Done processing!")

42375
42375
1720
1720
1335
1335
1335
0
Done processing!


In [5]:
# write train.csv
with open('train_validation.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(["Set", "Partner", "Class", "Count"])
    
    for index in range(0, len(set)):
        spamwriter.writerow([set[index], partner_values[index], class_values[index], count[index]])

TypeError: object of type 'type' has no len()

In [6]:
image_size = [2000]
for s in array_image_size_label_1:
    flag = 0
    for s1 in image_size:
        if s == s1:
            flag = 1
            break
    if flag == 0:
        image_size.append(s)
print(image_size)

[2000, '2160']


In [7]:
new_array_image_path_label_1 = []

for index in range(0, len(array_image_path_label_1)):
    image_path = array_image_path_label_1[index]
    image_path = image_path.split("\\")
    
    new_image_path = "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots"
    flag = -1
    for path in image_path:
        if flag != -1:
            new_image_path = new_image_path + "/" + str(path)
        if path == "2019_07_11_JUMP-CP-pilots":
            flag = 1
    new_array_image_path_label_1.append(str(new_image_path))
    
print(len(new_array_image_path_label_1))

1720


In [8]:
new_array_image_path_label_3 = []

for index in range(0, len(image_path_label_3)):
    image_path = image_path_label_3[index]
    image_path = image_path.split("\\")
    
    new_image_path = "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots"
    flag = -1
    for path in image_path:
        if flag != -1:
            new_image_path = new_image_path + "/" + str(path)
        if path == "2019_07_11_JUMP-CP-pilots":
            flag = 1
    new_array_image_path_label_3.append(str(new_image_path))
    
print(len(new_array_image_path_label_3))

NameError: name 'image_path_label_3' is not defined

In [9]:
# write train.csv
with open('../data/qc_1/train.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(["Image_Path", "Label", "Image_Size"])
    
    for index in range(0, len(new_array_image_path)):
        spamwriter.writerow([new_array_image_path[index], array_labels[index], array_image_size[index]])
        
    for index in range(0, len(image_path_label_3)):
        spamwriter.writerow([image_path_label_3[index], labels_3[index], image_size_label_3[index]])

FileNotFoundError: [Errno 2] No such file or directory: '../data/qc_1/train.csv'

In [10]:
# write train.csv
with open('qcdata_1channel/train_debris.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(["Image_Path", "Label"])
    
    for index in range(0, len(new_array_image_path)):
        spamwriter.writerow([new_array_image_path[index], "0"])

FileNotFoundError: [Errno 2] No such file or directory: 'qcdata_1channel/train_debris.csv'

In [22]:
# read csv files and obtain frequency for each class
array_labels = []
array_image_path_dna = []
array_image_path_rna = []
array_image_path_er = []
array_image_path_mito = []
array_image_path_agp = []
array_image_width = []
array_image_height = []
names_needed = []
for file in files:
    flag = -1
    flag_good = 1
    flag_debris = -1
    flag_empty = -1
    flag_saturated = -1
    flag_blurry = -1
    flag_dead = -1
    flag_halo = -1
    flag_fdebris = -1
    flag_focus = -1
    flag_image_width = -1
    flag_image_height = -1
    flag_image_path_dna = -1
    flag_image_path_rna = -1
    flag_image_path_er = -1
    flag_image_path_mito = -1
    flag_image_path_agp = -1
    
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                array = row
                if "UserAnnotation_Focus" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Focus":
                            flag_focus = index
                if "UserAnnotation_Scratch" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Scratch":
                            flag_saturated = index
                if "UserAnnotation_Halo" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Halo":
                            flag_halo = index
                if "UserAnnotation_Blurry" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Blurry":
                            flag_blurry = index
                if "UserAnnotation_Debris" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Debris":
                            flag_debris = index
                if "UserAnnotation_Empty" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Empty":
                            flag_empty = index
                if "UserAnnotation_Saturated" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Saturated":
                            flag_saturated = index
                if "UserAnnotation_Ded" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_Ded":
                            flag_dead = index
                if "UserAnnotation_NonFluoDebris" in array:
                    for index in range(0, len(array)):
                        if array[index] == "UserAnnotation_NonFluoDebris":
                            flag_fdebris = index
                if "Image_Width_OrigDNA" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_Width_OrigDNA":
                            flag_image_width = index
                if "Image_Height_OrigDNA" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_Height_OrigDNA":
                            flag_image_height = index
                if "Image_URL_OrigDNA" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigDNA":
                            flag_image_path_dna = index
                if "Image_URL_OrigRNA" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigRNA":
                            flag_image_path_rna = index
                if "Image_URL_OrigER" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigER":
                            flag_image_path_er = index
                if "Image_URL_OrigMito" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigMito":
                            flag_image_path_mito = index
                if "Image_URL_OrigAGP" in array:
                    for index in range(0, len(array)):
                        if array[index] == "Image_URL_OrigAGP":
                            flag_image_path_agp = index
                flag = 1
            else:
                array = row
                
                # check multi-class and remove them
                check_multi_class = 0
                if flag_focus != -1 and float(array[flag_focus]) > 0:
                    check_multi_class += 1
                if flag_blurry != -1 and float(array[flag_blurry]) > 0:
                    check_multi_class += 1
                if flag_debris != -1 and float(array[flag_debris]) > 0:
                    check_multi_class += 1
                if flag_fdebris != -1 and float(array[flag_fdebris]) > 0:
                    check_multi_class += 1
                if flag_saturated != -1 and float(array[flag_saturated]) > 0:
                    check_multi_class += 1
                if flag_halo != -1 and float(array[flag_halo]) > 0:
                    check_multi_class += 1
                if flag_empty != -1 and float(array[flag_empty]) > 0:
                    check_multi_class += 1
                if flag_dead != -1 and float(array[flag_dead]) > 0:
                    check_multi_class += 1
                if check_multi_class > 1:
                    continue
                
                check = -1
                is_saturated = -1
                label = 0
                if flag_blurry != -1 and float(array[flag_blurry]) > 0:
                    check += 1
                    label = 1
                if flag_debris != -1 and float(array[flag_debris]) > 0:
                    check += 1
                    label = 3
                if flag_fdebris != -1 and float(array[flag_fdebris]) > 0:
                    check += 1
                    label = 3
                if flag_saturated != -1 and float(array[flag_saturated]) > 0:
                    check += 1
                    is_saturated = 1
                if flag_halo != -1 and float(array[flag_halo]) > 0:
                    check += 1
                    is_saturated = 1
                if flag_focus != -1 and float(array[flag_focus]) > 0:
                    check += 1
                    is_saturated = 1
                if flag_empty != -1 and float(array[flag_empty]) > 0:
                    check += 1
                    label = 2
                if flag_dead != -1 and float(array[flag_dead]) > 0:
                    check += 1
                    label = 2
                    
                if is_saturated == -1:
                    if "BRO0117059-40x_20201013_155609" in str(array[flag_image_path_dna]):
                        s = str(array[flag_image_path_dna])
                        s = s.split("\\")
                        s = s[len(s)-1]
                        s = s.split(".")
                        names_needed.append(s[0])
                    if "BRO0117059-40x_20201013_155609" in str(array[flag_image_path_rna]):
                        s = str(array[flag_image_path_rna])
                        s = s.split("\\")
                        s = s[len(s)-1]
                        s = s.split(".")
                        names_needed.append(s[0])
                    if "BRO0117059-40x_20201013_155609" in str(array[flag_image_path_er]):
                        s = str(array[flag_image_path_er])
                        s = s.split("\\")
                        s = s[len(s)-1]
                        s = s.split(".")
                        names_needed.append(s[0])
                    if "BRO0117059-40x_20201013_155609" in str(array[flag_image_path_mito]):
                        s = str(array[flag_image_path_mito])
                        s = s.split("\\")
                        s = s[len(s)-1]
                        s = s.split(".")
                        names_needed.append(s[0])
                    if "BRO0117059-40x_20201013_155609" in str(array[flag_image_path_agp]):
                        s = str(array[flag_image_path_agp])
                        s = s.split("\\")
                        s = s[len(s)-1]
                        s = s.split(".")
                        names_needed.append(s[0])
                    
                    array_image_width.append(str(array[flag_image_width]))
                    array_image_height.append(str(array[flag_image_height]))
                    array_image_path_dna.append(str(array[flag_image_path_dna]))
                    array_image_path_rna.append(str(array[flag_image_path_rna]))
                    array_image_path_er.append(str(array[flag_image_path_er]))
                    array_image_path_mito.append(str(array[flag_image_path_mito]))
                    array_image_path_agp.append(str(array[flag_image_path_agp]))
                    array_labels.append(str(label))
print("Done processing!")

Done processing!


In [23]:
print(len(array_image_width))
print(len(array_image_height))
print(len(array_image_path_dna))
print(len(array_image_path_rna))
print(len(array_image_path_er))
print(len(array_image_path_mito))
print(len(array_image_path_agp))

54859
54859
54859
54859
54859
54859
54859


In [24]:
new_array_image_path_dna = []
new_array_image_path_rna = []
new_array_image_path_er = []
new_array_image_path_mito = []
new_array_image_path_agp = []

for index in range(0, len(array_image_path_dna)):
    image_path = array_image_path_dna[index]
    image_path = image_path.split("\\")
    
    new_image_path = "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots"
    flag = -1
    for path in image_path:
        if flag != -1:
            new_image_path = new_image_path + "/" + str(path)
    
        if path == "2019_07_11_JUMP-CP-pilots":
            flag = 1
    new_array_image_path_dna.append(str(new_image_path))


for index in range(0, len(array_image_path_rna)):
    image_path = array_image_path_rna[index]
    image_path = image_path.split("\\")
    
    new_image_path = "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots"
    flag = -1
    for path in image_path:
        if flag != -1:
            new_image_path = new_image_path + "/" + str(path)
    
        if path == "2019_07_11_JUMP-CP-pilots":
            flag = 1
    new_array_image_path_rna.append(str(new_image_path))
    
    
for index in range(0, len(array_image_path_er)):
    image_path = array_image_path_er[index]
    image_path = image_path.split("\\")
    
    new_image_path = "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots"
    flag = -1
    for path in image_path:
        if flag != -1:
            new_image_path = new_image_path + "/" + str(path)
    
        if path == "2019_07_11_JUMP-CP-pilots":
            flag = 1
    new_array_image_path_er.append(str(new_image_path))
    
    
for index in range(0, len(array_image_path_mito)):
    image_path = array_image_path_mito[index]
    image_path = image_path.split("\\")
    
    new_image_path = "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots"
    flag = -1
    for path in image_path:
        if flag != -1:
            new_image_path = new_image_path + "/" + str(path)
    
        if path == "2019_07_11_JUMP-CP-pilots":
            flag = 1
    new_array_image_path_mito.append(str(new_image_path))
    

for index in range(0, len(array_image_path_agp)):
    image_path = array_image_path_agp[index]
    image_path = image_path.split("\\")
    
    new_image_path = "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots"
    flag = -1
    for path in image_path:
        if flag != -1:
            new_image_path = new_image_path + "/" + str(path)
    
        if path == "2019_07_11_JUMP-CP-pilots":
            flag = 1
    new_array_image_path_agp.append(str(new_image_path))
    
    
print(len(new_array_image_path_agp))
print(len(new_array_image_path_mito))
print(len(new_array_image_path_er))
print(len(new_array_image_path_rna))
print(len(new_array_image_path_dna))

54859
54859
54859
54859
54859


In [25]:
# write train.csv
with open('../data/qc_5/train.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(["Image_Path_DNA", "Image_Path_RNA", "Image_Path_ER", "Image_Path_Mito", "Image_Path_AGP", "Label", "Image_Width", "Image_Height"])
    
    for index in range(0, len(array_image_width)):
        spamwriter.writerow([new_array_image_path_dna[index], new_array_image_path_rna[index], new_array_image_path_er[index], new_array_image_path_mito[index], new_array_image_path_agp[index], array_labels[index], array_image_width[index], array_image_height[index]])

In [27]:
# generate train.csv and val.csv
files = glob.glob("../data/qc_1/train.csv")
image_path_label_0 = []
labels_0 = []
image_size_label_0 = []
image_path_label_1 = []
labels_1 = []
image_size_label_1 = []
image_path_label_2 = []
labels_2 = []
image_size_label_2 = []
image_path_label_3 = []
labels_3 = []
image_size_label_3 = []
prev_file = "-1"
count_file = 0

for file in files:
    flag = -1
    
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                flag = 1
            else:
                array = row
                
                if file == "MyTrainingSet_4channel_urls.csv" or file == "MyTrainingSet_5channel_urls.csv" or file == "MyTrainingSet_6channel_urls.csv":
                    continue
                    
                if "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/2020_11_24_Scope1_Nikon/" in array[0]:
                    array[0] = array[0].replace("/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/2020_11_24_Scope1_Nikon/", "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/2020_11_24_Scope1_Nikon/tifs/")
                    file_list = array[0].split("/")
                    curr_file = file_list[len(file_list)-1]
                    curr_file = curr_file.replace("%20", " ")
                    curr_file = curr_file.replace("%2C", ",")
                    
                    if prev_file == curr_file and count == 1:
                        curr_file = curr_file[:len(curr_file)-4] + "_OrigDNA.tiff"
                        count += 1
                    elif prev_file == curr_file and count == 2:
                        curr_file = curr_file[:len(curr_file)-4] + "_OrigMito.tiff"
                        count += 1
                    elif prev_file == curr_file and count == 3:
                        curr_file = curr_file[:len(curr_file)-4] + "_OrigER.tiff"
                        count += 1
                    elif prev_file != curr_file:
                        prev_file = curr_file
                        curr_file = curr_file[:len(curr_file)-4] + "_OrigAGP.tiff"
                        count = 1
                    array[0] = "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/2020_11_24_Scope1_Nikon/tifs/" + str(file_list[len(file_list)-2]) + "/" + curr_file
                
                if "Nikon" in array[0]:
                    continue
                
                if int(array[1]) == 2:
                    image_path_label_2.append(str(array[0]))
                    labels_2.append(str(array[1]))
                    image_size_label_2.append(str(array[2]))
                    
                if int(array[1]) == 0:
                    image_path_label_0.append(str(array[0]))
                    labels_0.append(str(array[1]))
                    image_size_label_0.append(str(array[2]))
                    
                if int(array[1]) == 1:
                    image_path_label_1.append(str(array[0]))
                    labels_1.append(str(array[1]))
                    image_size_label_1.append(str(array[2]))
                    
                if int(array[1]) == 3:
                    image_path_label_3.append(str(array[0]))
                    labels_3.append(str(array[1]))
                    image_size_label_3.append(str(array[2]))
                    
print(len(labels_0))
print(len(labels_1))
print(len(labels_2))
print(len(labels_3))

134678
600
2012
745


In [33]:
count_good = 0
count_blurry = 0
count_empty = 0
count_debris = 0
for index in range(0, len(image_size_label_0)):
    if int(image_size_label_0[index]) == 5120:
        count_good += 1
        
for index in range(0, len(image_size_label_1)):
    if int(image_size_label_1[index]) == 5120:
        count_blurry += 1
        
for index in range(0, len(image_size_label_2)):
    if int(image_size_label_2[index]) == 5120:
        count_empty += 1
        
for index in range(0, len(image_size_label_3)):
    if int(image_size_label_3[index]) == 5120:
        count_debris += 1
        
        
print(count_good)
print(count_blurry)
print(count_empty)
print(count_debris)

23898
600
252
537


In [16]:
# generate train.csv and val.csv
files = glob.glob("../My*")
image_path_label_3 = []
labels_3 = []
image_size_label_3 = []
prev_file = "-1"
count_file = 0

for file in files:
    flag = -1
    index_dna = -1
    index_rna = -1
    index_er = -1
    index_mito = -1
    index_agp = -1
    index_actin = -1
    index_golgi = -1
    index_size = -1
    image_size = -1
    print(file)
    
    if file == "../MyTrainingSet_6channel_urls.csv":
        continue
    
    #if "MyTrainingSet_4channel" in file:
    #    image_size = 1608
    #elif "MyTrainingSet_6channel" in file:
    #    image_size = 5120
    
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                if "Image_URL_OrigAGP" in row:
                    for index in range(0, len(row)):
                        if row[index] == "Image_URL_OrigAGP":
                            index_agp = index
                if "Image_URL_OrigMito" in row:
                    for index in range(0, len(row)):
                        if row[index] == "Image_URL_OrigMito":
                            index_mito = index
                if "Image_URL_OrigER" in row:
                    for index in range(0, len(row)):
                        if row[index] == "Image_URL_OrigER":
                            index_er = index
                if "Image_URL_OrigRNA" in row:
                    for index in range(0, len(row)):
                        if row[index] == "Image_URL_OrigRNA":
                            index_rna = index
                if "Image_URL_OrigDNA" in row:
                    for index in range(0, len(row)):
                        if row[index] == "Image_URL_OrigDNA":
                            index_dna = index
                if "Image_URL_OrigActin" in row:
                    for index in range(0, len(row)):
                        if row[index] == "Image_URL_OrigActin":
                            index_actin = index
                if "Image_URL_OrigGolgi" in row:
                    for index in range(0, len(row)):
                        if row[index] == "Image_URL_OrigGolgi":
                            index_golgi = index
                if "Image_ImageQuality_TotalArea_OrigDNA.x" in row:
                    for index in range(0, len(row)):
                        if row[index] == "Image_ImageQuality_TotalArea_OrigDNA.x":
                            index_size = index
                flag = 1
            else:
                array = row
                    
                '''
                if "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/2020_11_24_Scope1_Nikon/" in array[0]:
                    array[0] = array[0].replace("/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/2020_11_24_Scope1_Nikon/", "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/2020_11_24_Scope1_Nikon/tifs/")
                    file_list = array[0].split("/")
                    curr_file = file_list[len(file_list)-1]
                    curr_file = curr_file.replace("%20", " ")
                    curr_file = curr_file.replace("%2C", ",")
                    
                    if prev_file == curr_file and count == 1:
                        curr_file = curr_file[:len(curr_file)-4] + "_OrigDNA.tiff"
                        count += 1
                    elif prev_file == curr_file and count == 2:
                        curr_file = curr_file[:len(curr_file)-4] + "_OrigMito.tiff"
                        count += 1
                    elif prev_file == curr_file and count == 3:
                        curr_file = curr_file[:len(curr_file)-4] + "_OrigER.tiff"
                        count += 1
                    elif prev_file != curr_file:
                        prev_file = curr_file
                        curr_file = curr_file[:len(curr_file)-4] + "_OrigAGP.tiff"
                        count = 1
                    array[0] = "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/2020_11_24_Scope1_Nikon/tifs/" + str(file_list[len(file_list)-2]) + "/" + curr_file
                '''
                image_size = int(math.sqrt(int(array[index_size])))
                
                
                if array[3] == "DNA":
                    if "Nikon" in array[index_dna]:
                        continue
                        
                    if image_size < 2500:
                        image_path_label_3.append(str(array[index_dna]))
                        labels_3.append("3")
                        image_size_label_3.append(str(int(math.sqrt(int(array[index_size])))))
                elif array[3] == "RNA":
                    if "Nikon" in array[index_rna]:
                        continue
                        
                    if image_size < 2500:
                        image_path_label_3.append(str(array[index_rna]))
                        labels_3.append("3")
                        image_size_label_3.append(str(int(math.sqrt(int(array[index_size])))))
                elif array[3] == "Mito":
                    if "Nikon" in array[index_mito]:
                        continue
                        
                    if image_size < 2500:
                        image_path_label_3.append(str(array[index_mito]))
                        labels_3.append("3")
                        image_size_label_3.append(str(int(math.sqrt(int(array[index_size])))))
                elif array[3] == "ER":
                    if "Nikon" in array[index_er]:
                        continue
                        
                    if image_size < 2500:
                        image_path_label_3.append(str(array[index_er]))
                        labels_3.append("3")
                        image_size_label_3.append(str(int(math.sqrt(int(array[index_size])))))
                elif array[3] == "AGP":
                    if "Nikon" in array[index_agp]:
                        continue
                        
                    if image_size < 2500:
                        image_path_label_3.append(str(array[index_agp]))
                        labels_3.append("3")
                        image_size_label_3.append(str(int(math.sqrt(int(array[index_size])))))
                elif array[3] == "Actin":
                    if "Nikon" in array[index_actin]:
                        continue
                        
                    if image_size < 2500:
                        image_path_label_3.append(str(array[index_actin]))
                        labels_3.append("3")
                        image_size_label_3.append(str(int(math.sqrt(int(array[index_size])))))
                elif array[3] == "Golgi":
                    if "Nikon" in array[index_golgi]:
                        continue
                        
                    if image_size < 2500:
                        image_path_label_3.append(str(array[index_golgi]))
                        labels_3.append("3")
                        image_size_label_3.append(str(int(math.sqrt(int(array[index_size])))))

for index in range(0, len(image_path_label_3)):
    path = image_path_label_3[index]
    path = path.replace("s3://jump-cellpainting/projects/2019_07_11_JUMP-CP-pilots/", "/dgx1nas1/cellpainting-datasets/2019_07_11_JUMP_CP_pilots/")
    image_path_label_3[index] = path
    

#print(len(labels_0))
#print(len(labels_1))
#print(len(labels_2))
print(len(labels_3))

../MyTrainingSet_4channel_urls.csv
../MyTrainingSet_6channel_urls.csv
../MyTrainingSet_5channel_urls.csv
208


In [20]:
count_debris = 0
for index in range(0, len(image_size_label_3)):
    if int(image_size_label_3[index]) == 1608:
        count_debris += 1
        
print(count_debris)

0


In [257]:
# generate train.csv and val.csv
train_label_0 = []
train_label_1 = []
train_label_2 = []
train_label_3 = []

val_label_0 = []
val_label_1 = []
val_label_2 = []
val_label_3 = []

files = glob.glob("qcdata_1channel/*")
image_path_label_0 = []
labels_0 = []
image_path_label_1 = []
labels_1 = []
image_path_label_2 = []
labels_2 = []
image_path_label_3 = []
labels_3 = []

for file in files:
    flag = -1
    
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                flag = 1
            else:
                array = row
                
                if str(array[0]).find(".nd2") != -1:
                    continue
                
                if file == "qcdata_1channel/train_debris.csv":
                    array[1] = "3"
                
                if int(array[1]) == 2:
                    image_path_label_2.append(str(array[0]))
                    labels_2.append(str(array[1]))
                    
                if int(array[1]) == 3:
                    image_path_label_3.append(str(array[0]))
                    labels_3.append(str(array[1]))
                    
                if int(array[1]) == 0:
                    image_path_label_0.append(str(array[0]))
                    labels_0.append(str(array[1]))
                    
                if int(array[1]) == 1:
                    image_path_label_1.append(str(array[0]))
                    labels_1.append(str(array[1]))
                    
print(len(labels_0))
print(len(labels_1))
print(len(labels_2))
print(len(labels_3))

134678
600
2012
3549


In [346]:
train_label_0 = []
train_label_1 = []
train_label_2 = []
train_label_3 = []

val_label_0 = []
val_label_1 = []
val_label_2 = []
val_label_3 = []

In [329]:
print(len(new_array_image_path_label_3))

208


In [310]:
n_array_image_path_label_0 = []
for index in range(0, len(new_array_image_path_label_0)):
    flag = 0
    for index1 in range(0, len(new_array_image_path_label_0)):
        if index != index1:
            if new_array_image_path_label_0[index] == new_array_image_path_label_0[index1]:
                flag = 1
                break
    if flag == 0:
        n_array_image_path_label_0.append(new_array_image_path_label_0[index])
print(len(n_array_image_path_label_0))

KeyboardInterrupt: 

In [271]:
n_array_image_path_label_1 = []
for index in range(0, len(new_array_image_path_label_1)):
    if "Nikon" in new_array_image_path_label_1[index]:
        continue
    n_array_image_path_label_1.append(new_array_image_path_label_1[index])
print(len(n_array_image_path_label_1))

10


In [349]:
for file in val_label_1:
    if "Nikon" in file:
        print(file)
        break

In [347]:
# for blur class
n = 40
index = np.random.choice(len(new_array_image_path_label_1), n, replace=False)

for index1 in range(0, len(index)):
    val_label_1.append(new_array_image_path_label_1[index[index1]])

index_train = []
for index1 in range(0, len(new_array_image_path_label_1)):
    flag = -1
    for index2 in range(0, len(index)):
        if index[index2] == index1:
            flag = 1
            break
    if flag == -1:
        index_train.append(index1)
        

for index1 in range(0, len(index_train)):
    if index1%5 == 0 and len(train_label_1) < 100:
        train_label_1.append(new_array_image_path_label_1[index_train[index1]])

print(len(train_label_1))
print(len(val_label_1))

100
40


In [118]:
# for good class
n = 40
index = np.random.choice(len(image_path_label_1), n, replace=False)

for index1 in range(0, len(index)):
    val_label_1.append(image_path_label_1[index[index1]])

index_train = []
for index1 in range(0, len(labels_1)):
    flag = -1
    for index2 in range(0, len(index)):
        if index[index2] == index1:
            flag = 1
            break
    if flag == -1:
        index_train.append(index1)
        

for index1 in range(0, len(index_train)):
    train_label_1.append(image_path_label_1[index_train[index1]])

print(len(train_label_1))
print(len(val_label_1))

500
100


In [119]:
# for empty class
n = 100
index = np.random.choice(len(image_path_label_2), n, replace=False)

for index1 in range(0, len(index)):
    val_label_2.append(image_path_label_2[index[index1]])

index_train = []
for index1 in range(0, len(labels_2)):
    flag = -1
    for index2 in range(0, len(index)):
        if index[index2] == index1:
            flag = 1
            break
    if flag == -1:
        index_train.append(index1)
        

for index1 in range(0, len(index_train)):
    train_label_2.append(image_path_label_2[index_train[index1]])

print(len(train_label_2))
print(len(val_label_2))

1912
100


In [120]:
# for debris class
n = 100
index = np.random.choice(len(image_path_label_3), n, replace=False)

for index1 in range(0, len(index)):
    val_label_3.append(image_path_label_3[index[index1]])

index_train = []
for index1 in range(0, len(labels_3)):
    flag = -1
    for index2 in range(0, len(index)):
        if index[index2] == index1:
            flag = 1
            break
    if flag == -1:
        index_train.append(index1)
        

for index1 in range(0, len(index_train)):
    train_label_3.append(image_path_label_3[index_train[index1]])

print(len(train_label_3))
print(len(val_label_3))

645
100


In [35]:
# generate train.csv and val.csv
train_dna_label_0 = []
train_rna_label_0 = []
train_er_label_0 = []
train_mito_label_0 = []
train_agp_label_0 = []
train_dna_label_1 = []
train_rna_label_1 = []
train_er_label_1 = []
train_mito_label_1 = []
train_agp_label_1 = []
train_dna_label_2 = []
train_rna_label_2 = []
train_er_label_2 = []
train_mito_label_2 = []
train_agp_label_2 = []
train_dna_label_3 = []
train_rna_label_3 = []
train_er_label_3 = []
train_mito_label_3 = []
train_agp_label_3 = []

val_dna_label_0 = []
val_rna_label_0 = []
val_er_label_0 = []
val_mito_label_0 = []
val_agp_label_0 = []
val_dna_label_1 = []
val_rna_label_1 = []
val_er_label_1 = []
val_mito_label_1 = []
val_agp_label_1 = []
val_dna_label_2 = []
val_rna_label_2 = []
val_er_label_2 = []
val_mito_label_2 = []
val_agp_label_2 = []
val_dna_label_3 = []
val_rna_label_3 = []
val_er_label_3 = []
val_mito_label_3 = []
val_agp_label_3 = []

files = glob.glob("../data/qc_5/train.csv")
image_path_dna_label_0 = []
image_path_rna_label_0 = []
image_path_er_label_0 = []
image_path_mito_label_0 = []
image_path_agp_label_0 = []
labels_0 = []
image_path_dna_label_1 = []
image_path_rna_label_1 = []
image_path_er_label_1 = []
image_path_mito_label_1 = []
image_path_agp_label_1 = []
labels_1 = []
image_path_dna_label_2 = []
image_path_rna_label_2 = []
image_path_er_label_2 = []
image_path_mito_label_2 = []
image_path_agp_label_2 = []
labels_2 = []
image_path_dna_label_3 = []
image_path_rna_label_3 = []
image_path_er_label_3 = []
image_path_mito_label_3 = []
image_path_agp_label_3 = []
labels_3 = []

for file in files:
    flag = -1
    
    if "train" not in file:
        continue
    
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                flag = 1
            else:
                array = row
                
                if int(array[5]) == 2:
                    image_path_dna_label_2.append(str(array[0]))
                    image_path_rna_label_2.append(str(array[1]))
                    image_path_er_label_2.append(str(array[2]))
                    image_path_mito_label_2.append(str(array[3]))
                    image_path_agp_label_2.append(str(array[4]))
                    labels_2.append(str(array[5]))
                    
                if int(array[5]) == 3:
                    image_path_dna_label_3.append(str(array[0]))
                    image_path_rna_label_3.append(str(array[1]))
                    image_path_er_label_3.append(str(array[2]))
                    image_path_mito_label_3.append(str(array[3]))
                    image_path_agp_label_3.append(str(array[4]))
                    labels_3.append(str(array[5]))
                    
                if int(array[5]) == 0:
                    image_path_dna_label_0.append(str(array[0]))
                    image_path_rna_label_0.append(str(array[1]))
                    image_path_er_label_0.append(str(array[2]))
                    image_path_mito_label_0.append(str(array[3]))
                    image_path_agp_label_0.append(str(array[4]))
                    labels_0.append(str(array[5]))
                
                if int(array[5]) == 1:
                    image_path_dna_label_1.append(str(array[0]))
                    image_path_rna_label_1.append(str(array[1]))
                    image_path_er_label_1.append(str(array[2]))
                    image_path_mito_label_1.append(str(array[3]))
                    image_path_agp_label_1.append(str(array[4]))
                    labels_1.append(str(array[5]))
                    
print(len(labels_0))
print(len(labels_1))
print(len(labels_2))
print(len(labels_3))

52999
346
1136
378


In [36]:
# for good class
n = 100
index = np.random.choice(len(image_path_dna_label_0), n, replace=False)

for index1 in range(0, len(index)):
    val_dna_label_0.append(image_path_dna_label_0[index[index1]])
    val_rna_label_0.append(image_path_rna_label_0[index[index1]])
    val_er_label_0.append(image_path_er_label_0[index[index1]])
    val_mito_label_0.append(image_path_mito_label_0[index[index1]])
    val_agp_label_0.append(image_path_agp_label_0[index[index1]])


index_train = []
for index1 in range(0, len(labels_0)):
    flag = -1
    for index2 in range(0, len(index)):
        if index[index2] == index1:
            flag = 1
            break
    if flag == -1:
        index_train.append(index1)
        

for index1 in range(0, len(index_train)):
    if index1%5 == 0:
        train_dna_label_0.append(image_path_dna_label_0[index_train[index1]])
        train_rna_label_0.append(image_path_rna_label_0[index_train[index1]])
        train_er_label_0.append(image_path_er_label_0[index_train[index1]])
        train_mito_label_0.append(image_path_mito_label_0[index_train[index1]])
        train_agp_label_0.append(image_path_agp_label_0[index_train[index1]])

print(len(train_dna_label_0))
print(len(val_dna_label_0))

10580
100


In [37]:
# for empty class
n = 100
index = np.random.choice(len(image_path_dna_label_2), n, replace=False)

for index1 in range(0, len(index)):
    val_dna_label_2.append(image_path_dna_label_2[index[index1]])
    val_rna_label_2.append(image_path_rna_label_2[index[index1]])
    val_er_label_2.append(image_path_er_label_2[index[index1]])
    val_mito_label_2.append(image_path_mito_label_2[index[index1]])
    val_agp_label_2.append(image_path_agp_label_2[index[index1]])


index_train = []
for index1 in range(0, len(labels_2)):
    flag = -1
    for index2 in range(0, len(index)):
        if index[index2] == index1:
            flag = 1
            break
    if flag == -1:
        index_train.append(index1)
        

for index1 in range(0, len(index_train)):
    train_dna_label_2.append(image_path_dna_label_2[index_train[index1]])
    train_rna_label_2.append(image_path_rna_label_2[index_train[index1]])
    train_er_label_2.append(image_path_er_label_2[index_train[index1]])
    train_mito_label_2.append(image_path_mito_label_2[index_train[index1]])
    train_agp_label_2.append(image_path_agp_label_2[index_train[index1]])

print(len(train_dna_label_2))
print(len(val_dna_label_2))

1036
100


In [38]:
# for empty class
n = 100
index = np.random.choice(len(image_path_dna_label_3), n, replace=False)

for index1 in range(0, len(index)):
    val_dna_label_3.append(image_path_dna_label_3[index[index1]])
    val_rna_label_3.append(image_path_rna_label_3[index[index1]])
    val_er_label_3.append(image_path_er_label_3[index[index1]])
    val_mito_label_3.append(image_path_mito_label_3[index[index1]])
    val_agp_label_3.append(image_path_agp_label_3[index[index1]])


index_train = []
for index1 in range(0, len(labels_3)):
    flag = -1
    for index2 in range(0, len(index)):
        if index[index2] == index1:
            flag = 1
            break
    if flag == -1:
        index_train.append(index1)
        

for index1 in range(0, len(index_train)):
    train_dna_label_3.append(image_path_dna_label_3[index_train[index1]])
    train_rna_label_3.append(image_path_rna_label_3[index_train[index1]])
    train_er_label_3.append(image_path_er_label_3[index_train[index1]])
    train_mito_label_3.append(image_path_mito_label_3[index_train[index1]])
    train_agp_label_3.append(image_path_agp_label_3[index_train[index1]])

print(len(train_dna_label_3))
print(len(val_dna_label_3))

278
100


In [39]:
# for empty class
n = 100
index = np.random.choice(len(image_path_dna_label_1), n, replace=False)

for index1 in range(0, len(index)):
    val_dna_label_1.append(image_path_dna_label_1[index[index1]])
    val_rna_label_1.append(image_path_rna_label_1[index[index1]])
    val_er_label_1.append(image_path_er_label_1[index[index1]])
    val_mito_label_1.append(image_path_mito_label_1[index[index1]])
    val_agp_label_1.append(image_path_agp_label_1[index[index1]])


index_train = []
for index1 in range(0, len(labels_1)):
    flag = -1
    for index2 in range(0, len(index)):
        if index[index2] == index1:
            flag = 1
            break
    if flag == -1:
        index_train.append(index1)
        

for index1 in range(0, len(index_train)):
    train_dna_label_1.append(image_path_dna_label_1[index_train[index1]])
    train_rna_label_1.append(image_path_rna_label_1[index_train[index1]])
    train_er_label_1.append(image_path_er_label_1[index_train[index1]])
    train_mito_label_1.append(image_path_mito_label_1[index_train[index1]])
    train_agp_label_1.append(image_path_agp_label_1[index_train[index1]])

print(len(train_dna_label_1))
print(len(val_dna_label_1))

246
100


In [40]:
train_dna = []
train_rna = []
train_er = []
train_mito = []
train_agp = []
train_labels = []

for index in range(0, len(train_dna_label_0)):
    train_dna.append(train_dna_label_0[index])
    train_rna.append(train_rna_label_0[index])
    train_er.append(train_er_label_0[index])
    train_mito.append(train_mito_label_0[index])
    train_agp.append(train_agp_label_0[index])
    train_labels.append("0")
    
for index in range(0, len(train_dna_label_1)):
    train_dna.append(train_dna_label_1[index])
    train_rna.append(train_rna_label_1[index])
    train_er.append(train_er_label_1[index])
    train_mito.append(train_mito_label_1[index])
    train_agp.append(train_agp_label_1[index])
    train_labels.append("1")
    
for index in range(0, len(train_dna_label_2)):
    train_dna.append(train_dna_label_2[index])
    train_rna.append(train_rna_label_2[index])
    train_er.append(train_er_label_2[index])
    train_mito.append(train_mito_label_2[index])
    train_agp.append(train_agp_label_2[index])
    train_labels.append("2")
    
for index in range(0, len(train_dna_label_3)):
    train_dna.append(train_dna_label_3[index])
    train_rna.append(train_rna_label_3[index])
    train_er.append(train_er_label_3[index])
    train_mito.append(train_mito_label_3[index])
    train_agp.append(train_agp_label_3[index])
    train_labels.append("3")
    

print(len(train_dna))
print(len(train_rna))
print(len(train_er))
print(len(train_mito))
print(len(train_agp))
print(len(train_labels))

12140
12140
12140
12140
12140
12140


In [41]:
val_dna = []
val_rna = []
val_er = []
val_mito = []
val_agp = []
val_labels = []

for index in range(0, len(val_dna_label_0)):
    val_dna.append(val_dna_label_0[index])
    val_rna.append(val_rna_label_0[index])
    val_er.append(val_er_label_0[index])
    val_mito.append(val_mito_label_0[index])
    val_agp.append(val_agp_label_0[index])
    val_labels.append("0")
    
for index in range(0, len(val_dna_label_1)):
    val_dna.append(val_dna_label_1[index])
    val_rna.append(val_rna_label_1[index])
    val_er.append(val_er_label_1[index])
    val_mito.append(val_mito_label_1[index])
    val_agp.append(val_agp_label_1[index])
    val_labels.append("1")
    
for index in range(0, len(val_dna_label_2)):
    val_dna.append(val_dna_label_2[index])
    val_rna.append(val_rna_label_2[index])
    val_er.append(val_er_label_2[index])
    val_mito.append(val_mito_label_2[index])
    val_agp.append(val_agp_label_2[index])
    val_labels.append("2")
    
for index in range(0, len(val_dna_label_3)):
    val_dna.append(val_dna_label_3[index])
    val_rna.append(val_rna_label_3[index])
    val_er.append(val_er_label_3[index])
    val_mito.append(val_mito_label_3[index])
    val_agp.append(val_agp_label_3[index])
    val_labels.append("3")
    

print(len(val_dna))
print(len(val_rna))
print(len(val_er))
print(len(val_mito))
print(len(val_agp))
print(len(val_labels))

400
400
400
400
400
400


In [42]:
# write final_train.csv
with open('../data/qc_5/f_train.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(["Image_Path_DNA", "Image_Path_RNA", "Image_Path_ER", "Image_Path_Mito", "Image_Path_AGP", "Label"])
    
    for index in range(0, len(train_dna)):
        spamwriter.writerow([train_dna[index], train_rna[index], train_er[index], train_mito[index], train_agp[index], train_labels[index]])

In [43]:
# write final_train.csv
with open('../data/qc_5/f_val.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(["Image_Path_DNA", "Image_Path_RNA", "Image_Path_ER", "Image_Path_Mito", "Image_Path_AGP", "Label"])
    
    for index in range(0, len(val_dna)):
        spamwriter.writerow([val_dna[index], val_rna[index], val_er[index], val_mito[index], val_agp[index], val_labels[index]])

In [121]:
train_path = []
train_labels = []

for index in range(0, len(train_label_0)):
    train_path.append(train_label_0[index])
    train_labels.append("0")

for index in range(0, len(train_label_1)):
    train_path.append(train_label_1[index])
    train_labels.append("1")    

for index in range(0, len(train_label_2)):
    train_path.append(train_label_2[index])
    train_labels.append("2")
    
for index in range(0, len(train_label_3)):
    train_path.append(train_label_3[index])
    train_labels.append("3")
    

print(len(train_path))
print(len(train_labels))

5128
5128


In [122]:
# write final_train.csv
with open('../data/qcdata_1channel/final_train.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(["Image_Path", "Label"])
    
    for index in range(0, len(train_path)):
        spamwriter.writerow([train_path[index], train_labels[index]])

In [123]:
val_path = []
val_labels = []

for index in range(0, len(val_label_0)):
    val_path.append(val_label_0[index])
    val_labels.append("0")
    
for index in range(0, len(val_label_1)):
    val_path.append(val_label_1[index])
    val_labels.append("1")
    
for index in range(0, len(val_label_2)):
    val_path.append(val_label_2[index])
    val_labels.append("2")
    
for index in range(0, len(val_label_3)):
    val_path.append(val_label_3[index])
    val_labels.append("3")
    

print(len(val_path))
print(len(val_labels))

400
400


In [124]:
# write final_train.csv
with open('../data/qcdata_1channel/final_val.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(["Image_Path", "Label"])
    
    for index in range(0, len(val_path)):
        spamwriter.writerow([val_path[index], val_labels[index]])

In [20]:
# generate more data augmentation samples
files = glob.glob("qcdata_5channel/*")
image_path_dna = []
image_path_rna = []
image_path_er = []
image_path_mito = []
image_path_agp = []
labels = []
for file in files:
    flag = -1
    
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                flag = 1
            else:
                array = row
                image_path_dna.append(str(array[0]))
                image_path_rna.append(str(array[1]))
                image_path_er.append(str(array[2]))
                image_path_mito.append(str(array[3]))
                image_path_agp.append(str(array[4]))
                labels.append(str(array[5]))
                
                #if int(array[5]) == 1 or int(array[5]) == 2:
                #    image_path_dna.append(str(array[0]))
                #    image_path_rna.append(str(array[1]))
                #    image_path_er.append(str(array[2]))
                #    image_path_mito.append(str(array[3]))
                #    image_path_agp.append(str(array[4]))
                #    labels.append(str(array[5]))
                    
print(len(image_path_dna))
print(len(image_path_rna))
print(len(image_path_er))
print(len(image_path_mito))
print(len(image_path_agp))
print(len(labels))

24591
24591
24591
24591
24591
24591


In [19]:
# write train.csv
with open('train.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow(["Image_Path_DNA", "Image_Path_RNA", "Image_Path_ER", "Image_Path_Mito", "Image_Path_AGP", "Label"])
    
    for index in range(0, len(image_path_dna)):
        spamwriter.writerow([image_path_dna[index], image_path_rna[index], image_path_er[index], image_path_mito[index], image_path_agp[index], labels[index]])